# Instalacion

In [ ]:
!pip install chromadb 

# [Inicializacion de chromadb](https://docs.trychroma.com/reference/Client)

Puedes crear una DB en memoria para pruebas creando un cliente Chroma DB sin configuraciones.

En nuestro caso, crearemos una base de datos persistente que se almacenará en el directorio "db/" y utilizaremos DuckDB en el backend.

Crea una instancia persistente de Chroma que se guarda en disco. Esto es útil para pruebas y desarrollo, pero no se recomienda para uso en producción.

Argumentos

* path: Directorio en el que se guardarán los datos de Chroma. Por defecto es "./chroma".
* tenant: El tenant que se utilizará para este cliente. Por defecto es el tenant predeterminado.
* database: La base de datos a utilizar para este cliente. Por defecto es la base de datos por defecto.

In [19]:
import chromadb
client = chromadb.PersistentClient(path="./data")

# collecciones

## Crear colleccion

Crea una nueva colección con el nombre y los metadatos dados. Args:
    name: El nombre de la colección a crear.
    metadatos: Metadatos opcionales para asociar a la colección.
    función_incrustar: Función opcional para incrustar documentos.
                        Si no se especifica, se utiliza la función de incrustación predeterminada.
    get_or_create: Si es True, devuelve la colección existente si existe.

Ejemplos:

    client.create_collection("mi_coleccion")
    # colección(nombre="mi_colección", metadatos={})

    client.create_collection("mi_coleccion", metadatos={"foo": "bar"})
    # collection(name="mi_coleccion", metadatos={"foo": "bar"})

In [7]:
collection = client.create_collection(name="Students")

### Crea una coleccion si no existe

In [8]:
collection = client.get_or_create_collection("Students")

## Modifica colecciones

* name: Actualiza el nombre de la coleccion. Optional.
* metadata: Actualiza los metadatos de la coleccion. Optional.



In [9]:
collection.modify(name="Students")

## Eliminar colecciones

* name: Nombre de la coleccion a eliminar.

In [6]:
client.delete_collection("Students")

## Lista las colecciones existentes

In [10]:
client.list_collections()

[Collection(name=Students)]

# Agregamos documentos

Las colleciones permiten almacenar documentos y metadata asociados a la coleccion

## Datos a guardar

In [11]:
student_info = """
Alexandra Thompson, estudiante de segundo curso de informática de 19 años con un promedio de 3,7,
es miembro de los clubes de programación y ajedrez, y en su tiempo libre disfruta de la pizza, la natación y el senderismo.
en su tiempo libre con la esperanza de trabajar en una empresa de tecnología después de graduarse en la Universidad de Washington.
"""

club_info = """
El club universitario de ajedrez ofrece a los estudiantes la oportunidad de reunirse y disfrutar jugando
el clásico juego de estrategia del ajedrez. Los miembros de todos los niveles son bienvenidos, desde principiantes que aprenden las reglas hasta jugadores de torneos experimentados.
desde principiantes que aprenden las reglas hasta jugadores experimentados en torneos. El club suele reunirse varias veces por semana para jugar partidas informales,
participar en torneos, analizar partidas de ajedrez famosas y mejorar las habilidades de los miembros.
"""

university_info = """
La Universidad de Washington, fundada en 1861 en Seattle, es una universidad pública de investigación
con más de 45.000 estudiantes repartidos en tres campus en Seattle, Tacoma y Bothell.
Es el buque insignia de las seis universidades públicas del estado de Washington,
UW abarca más de 500 edificios y 20 millones de pies cuadrados de espacio,
incluyendo uno de los mayores sistemas bibliotecarios del mundo.
"""

## Guardar datos

A diferencia de otras bases de datos vecotriales, chromaDB genera embeddings automaticamente usando el modelo de all-MiniLM-L6-v2, asi que solo tenemos que pasar los textos

In [12]:
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "infromacion del estudiante"},{"source": "informacion del club"},{'source':'informacion de la universidad'}],
    ids = ["id1", "id2", "id3"]
)

# Query

El siguiente es un ejemplo de query en colecciones, de esta forma podemos hacer busquedas sus argumentos son

* query_embeddings: Las incrustaciones de las que obtener los vecinos cercanos. Opcional.
* query_texts: Los textos del documento para obtener los vecinos cercanos. Opcional.
* query_images: Las imágenes para obtener los vecinos cercanos. Opcional.
* n_results: El número de vecinos a devolver para cada query_embedding o query_texts. Opcional.
* where: Un dict de tipo Where utilizado para filtrar los resultados por. Por ejemplo, {"$and": ["color" : "rojo", "precio": {"$gte": 4.20}]}. Opcional.

In [17]:
results = collection.query(
    query_texts=["¿Quien es Alexandra Thompson?"],
    n_results=2
)

results

{'ids': [['id1', 'id2']],
 'distances': [[0.9922355851517328, 1.3197836039520348]],
 'metadatas': [[{'source': 'infromacion del estudiante'},
   {'source': 'informacion del club'}]],
 'embeddings': None,
 'documents': [['\nAlexandra Thompson, estudiante de segundo curso de informática de 19 años con un promedio de 3,7,\nes miembro de los clubes de programación y ajedrez, y en su tiempo libre disfruta de la pizza, la natación y el senderismo.\nen su tiempo libre con la esperanza de trabajar en una empresa de tecnología después de graduarse en la Universidad de Washington.\n',
   '\nEl club universitario de ajedrez ofrece a los estudiantes la oportunidad de reunirse y disfrutar jugando\nel clásico juego de estrategia del ajedrez. Los miembros de todos los niveles son bienvenidos, desde principiantes que aprenden las reglas hasta jugadores de torneos experimentados.\ndesde principiantes que aprenden las reglas hasta jugadores experimentados en torneos. El club suele reunirse varias veces 